In [ ]:
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import tensorflow as tf
from sklearn.cluster import KMeans
from PIL import UnidentifiedImageError
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
DL_MODEL = ResNet50(weights='imagenet', include_top=False)

In [ ]:
batch_size = 1
n_clusters= 10
img_height = 224
img_width = 224
data_dir="data/animals10_small"
dataset = tf.keras.preprocessing.image_dataset_from_directory(data_dir, image_size=(img_height,img_width),
                                                               batch_size=batch_size,labels="inferred",crop_to_aspect_ratio=True)

In [ ]:
#batch 1
lista=[]
labels=[]
lista_img=[]
for x,y in dataset:
    labels.append(max(y.numpy()))
    lista_img.append(x)
    prepros=preprocess_input(x)
    prepros=DL_MODEL.predict(prepros)
    lista.append(prepros.flatten())
kmeans=KMeans(n_clusters=n_clusters,max_iter=1000,random_state=42,n_init=20).fit(lista)

In [ ]:
df=pd.DataFrame(kmeans.labels_,columns=["cluster"])
df["labels"]=labels

In [ ]:
df["foto_array"]=[img.numpy().reshape(224,224,3).astype("uint8") for img in lista_img]

In [ ]:
disp = ConfusionMatrixDisplay.from_predictions(df["labels"],df["cluster"])
_=disp.figure_.suptitle("Confusion Matrix")
cm=disp.confusion_matrix
# print(f"Confusion matrix:\n{disp.confusion_matrix}")

In [ ]:
def reorder_clusters(clusters,cm):
    new_clusters={}
    possible_clusters=list(range(cm.shape[0]))
    for idx,row in enumerate(cm):
        sorted_index=np.argsort(row)[::-1]
        for jdx in sorted_index:
            if jdx in possible_clusters:
                new_clusters[idx]=jdx
                break
        possible_clusters=list(filter(lambda item:item!=jdx,possible_clusters))
    new_clusters={v:k for k,v in new_clusters.items()}
    return [new_clusters[i] for i in clusters] 

In [ ]:
def order(clusters,cm):
    new_clusters={}
    dic={}
    trava=-1
    possible_clusters=[]
    #faz possible clusters com valores True
    [possible_clusters.append(True)for i in range(cm.shape[0])]
    #loop
    for idx, row in enumerate(cm):
        while(True):#trava caso o numero não ache de primeira
            try:
                max=np.argsort(row)[trava]#pega a id do item -x da row
                for kdx in range(cm.shape[0]):#checa se tem algum numero maior que ele na coluna
                    if cm[kdx][max]>row.max() and possible_clusters[kdx]:#se tiver e ele ainda não estiver sido usado(valor True na possible_clusters)
                        new_clusters[kdx]=max
                        trava-=1#trava muda de valor pra ver o proximo numero maior da row
                        possible_clusters[kdx]=False#usa o new_cluster
                        break#sai do loop do for(13)
                else:
                        new_clusters[idx]=max
                        trava=-1#reseta trava para a proxima row
                        possible_clusters[idx]=False
                        break#sai do loop do while(10) e avança pra proxima iteração do for(9)
            except:
                pass
    #limpa dic pra voltar
    dic={v:k for k,v in new_clusters.items()}
    return [dic[i] for i in clusters]

In [ ]:
def interface_cm(df,cm):
    try:
        return order(df.cluster,cm)
    except:
        print("reorder")
        return reorder_clusters(df.cluster,cm)

In [ ]:
disp = ConfusionMatrixDisplay.from_predictions(df["labels"],interface_cm(df,cm))
_=disp.figure_.suptitle("Confusion Matrix")
# print(f"Confusion matrix:\n{disp.confusion_matrix}")

In [ ]:
df["cluster"]=interface_cm(df,cm)
df[df.labels!=df.cluster]

In [ ]:
dic_nomes={0:"Butterfly",1:"Cat",2:"Chicken",3:"Cow",4:"Dog",5:"Elephant",6:"Horse",7:"Sheep",8:"Spider",9:"Squirrel"}

In [ ]:
n_img=10
erros=list(df[df.labels!=df.cluster].index)
feed=erros[:n_img]
for id in feed:
    fig,ax=plt.subplots(1,1)
    _=ax.imshow(df.loc[id,"foto_array"])
    label=df.loc[id,"labels"]
    pred=df.loc[id,"cluster"]
    _=ax.set_title(f"label: {dic_nomes[label]}   pred: {dic_nomes[pred]}")
    plt.axis("off")

In [ ]:
n_img=1
for id in dic_nomes:
    erros=list(df[(df.labels==id)&  (df.cluster==id)].index)
    feed=erros[:n_img]
    for ex in feed:
        fig,ax=plt.subplots(1,1)
        _=ax.imshow(df.loc[ex,"foto_array"])
        label=df.loc[ex,"labels"]
        pred=df.loc[ex,"cluster"]
        _=ax.set_title(f"label: {dic_nomes[label]}   pred: {dic_nomes[pred]}")
        plt.axis("off")